<a href="https://www.kaggle.com/code/olimsuyunboyev/backpack-challenge?scriptVersionId=248321158" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline
# from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
df = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e2/test.csv")
# train_extra = pd.read_csv("/kaggle/input/playground-series-s5e2/training_extra.csv")

In [4]:
# df = pd.concat([train, train_extra])

In [5]:
test.fillna(method='ffill', inplace=True)

<ipython-input-5-0ee31a326f49>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test.fillna(method='ffill', inplace=True)


In [6]:
df.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [7]:
df.isnull().sum()

id                         0
Brand                   9705
Material                8347
Size                    6595
Compartments               0
Laptop Compartment      7444
Waterproof              7050
Style                   7970
Color                   9950
Weight Capacity (kg)     138
Price                      0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    300000 non-null  int64  
 1   Brand                 290295 non-null  object 
 2   Material              291653 non-null  object 
 3   Size                  293405 non-null  object 
 4   Compartments          300000 non-null  float64
 5   Laptop Compartment    292556 non-null  object 
 6   Waterproof            292950 non-null  object 
 7   Style                 292030 non-null  object 
 8   Color                 290050 non-null  object 
 9   Weight Capacity (kg)  299862 non-null  float64
 10  Price                 300000 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 25.2+ MB


In [9]:
df = df.dropna()

In [10]:
test_ids = test['id']

In [11]:
# df = df.drop('id', axis=1)
# test = test.drop('id', axis=1)

In [12]:
df.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [13]:
df.isnull().sum()/df.shape[0]

id                      0.0
Brand                   0.0
Material                0.0
Size                    0.0
Compartments            0.0
Laptop Compartment      0.0
Waterproof              0.0
Style                   0.0
Color                   0.0
Weight Capacity (kg)    0.0
Price                   0.0
dtype: float64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246686 entries, 0 to 299999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    246686 non-null  int64  
 1   Brand                 246686 non-null  object 
 2   Material              246686 non-null  object 
 3   Size                  246686 non-null  object 
 4   Compartments          246686 non-null  float64
 5   Laptop Compartment    246686 non-null  object 
 6   Waterproof            246686 non-null  object 
 7   Style                 246686 non-null  object 
 8   Color                 246686 non-null  object 
 9   Weight Capacity (kg)  246686 non-null  float64
 10  Price                 246686 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 22.6+ MB


In [15]:
label = LabelEncoder()
df['Brand'] = label.fit_transform(df['Brand'].values)
df['Material'] = label.fit_transform(df['Material'].values)
df['Size'] = label.fit_transform(df['Size'].values)
df['Laptop Compartment'] = label.fit_transform(df['Laptop Compartment'].values)
df['Waterproof'] = label.fit_transform(df['Waterproof'].values)
df['Style'] = label.fit_transform(df['Style'].values)
df['Color'] = label.fit_transform(df['Color'].values)

In [16]:
label = LabelEncoder()
test['Brand'] = label.fit_transform(test['Brand'].values)
test['Material'] = label.fit_transform(test['Material'].values)
test['Size'] = label.fit_transform(test['Size'].values)
test['Laptop Compartment'] = label.fit_transform(test['Laptop Compartment'].values)
test['Waterproof'] = label.fit_transform(test['Waterproof'].values)
test['Style'] = label.fit_transform(test['Style'].values)
test['Color'] = label.fit_transform(test['Color'].values)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246686 entries, 0 to 299999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    246686 non-null  int64  
 1   Brand                 246686 non-null  int64  
 2   Material              246686 non-null  int64  
 3   Size                  246686 non-null  int64  
 4   Compartments          246686 non-null  float64
 5   Laptop Compartment    246686 non-null  int64  
 6   Waterproof            246686 non-null  int64  
 7   Style                 246686 non-null  int64  
 8   Color                 246686 non-null  int64  
 9   Weight Capacity (kg)  246686 non-null  float64
 10  Price                 246686 non-null  float64
dtypes: float64(3), int64(8)
memory usage: 22.6 MB


In [18]:
df.corrwith(df['Price']).abs().sort_values(ascending=False)

Price                   1.000000
Weight Capacity (kg)    0.015811
Brand                   0.007412
Color                   0.002963
Size                    0.002920
Waterproof              0.002248
id                      0.001807
Material                0.001536
Laptop Compartment      0.001490
Style                   0.000082
Compartments            0.000066
dtype: float64

# Correlyatsiya juda past! Boshqa regressiya modellaridan foydalanamiz

In [19]:
X = df.drop('Price', axis=1)
y = df['Price']

In [20]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=11)

In [22]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'n_estimators': [250, 300, 350],
#     'max_depth': [10, 15, 20],
#     'min_samples_split': [2, 4, 6],
#     'min_samples_leaf': [1, 2, 3]
# }

# grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
# grid_search.fit(X_train, y_train)

# print("Yakuniy eng yaxshi parametrlar:", grid_search.best_params_)


In [23]:
# Modelni yaratish va o'qitish
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    max_features='sqrt',
    bootstrap=True,
    random_state=42)

model.fit(X_train, y_train)

RandomForestRegressor(max_depth=15, max_features='sqrt', random_state=42)

In [24]:
# Test datasetida bashorat qilish
predictions = model.predict(X_test)

In [25]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("RMSE:", rmse)

RMSE: 38.865557165025365


In [26]:
test_prediction = model.predict(test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [27]:
submission = pd.DataFrame({
    'id': test_ids,
    'Price': test_prediction
})

submission.to_csv('_sample_submission.csv', index=False)

print("Submission fayli muvaffaqiyatli saqlandi!")

Submission fayli muvaffaqiyatli saqlandi!


In [28]:
print(f"X_test uzunligi: {len(X_test)}")
print(f"predictions uzunligi: {len(predictions)}")
print(f"test uzunligi: {len(test)}")

X_test uzunligi: 24669
predictions uzunligi: 24669
test uzunligi: 200000
